# **Task discription:**

---


Set-up a rough framework which simulates the reflections
of 2 (imaginative) forest types at 4 points within the vegetation period
(winter, spring, summer, autumn) and for 2 conditions: undisturbed and
disturbed.

---



# **Task preparation:**

---




##Prepared input (refer from): 

  - The PROSAIL model (Python-implementations): [link](https://github.com/robintw/ProSAIL)

  - Forest types: Temperate broad-leaved rainforest & Temperate dry conifer forest.
  - Leaf chemistry & structure properties (N, Cab, Car, Cbrown, Cw, Cm): [link](https://core.ac.uk/download/pdf/77234212.pdf)
  - (LIDF) Leaf Inclination Distribution Function (TypeLidf): [link](https://www.sciencedirect.com/science/article/abs/pii/S0168192312003036?via%3Dihub)
  - 4SAIL canopy structure parameters (LAI, hspot, tts, tto, psi).
    -  LAI (Leaf Area Index) at 4 points within the vegetation period (winter, spring, summer, autumn): [link](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0032155)
    - During all PROSAIL simulations the soil brightness parameter (psoil), which determines the moisture content of the soil, was kept constant at 0.5. The sun angle (tts) was set to 35° , he relative azimuth angle between sun and satellite sensor (psi) was set to 90°, and the observer angle (tto) was set to nadir (0°), resulting in a negligible effect of the hotspot size parameter, which was therefore kept constant at 0.01.


# **Task implimentation:**

---


In [ ]:
# Install the requirement package
!pip3 install gitpython

     |████████████████████████████████| 163kB 5.5MB/s 
     |████████████████████████████████| 71kB 5.9MB/s 


In [ ]:
import os
import sys
import logging
from collections import OrderedDict 

from git.repo.base import Repo

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable 
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, widgets, Layout
plt.style.use('seaborn')
%matplotlib inline

logger = logging.getLogger('')
logger.setLevel(logging.INFO)

original_dir = os.getcwd()

# Setup working directory where python scripts/packages, data and output results be stored. 
folder_name = "work"
prosail_git_package_url = "https://github.com/robintw/ProSAIL.git"

# Check if the working directory is available, if NOT --> the folder will be created!
try:
    os.makedirs(folder_name)
    logging.info('{} folder has been successfully created!'.format(folder_name))
except FileExistsError:
    logging.info('{} folder does exist!'.format(folder_name))
    pass

# Clone ProSAIL python package from the repository.
try:
  logging.info('ProSAIL package has successfully cloned from {} repository'.\
               format(prosail_git_package_url))
  Repo.clone_from(prosail_git_package_url, folder_name)
except:
  logging.warning('The repository {} is no longer available'\
                  ' Please check from other sources!'.
                  format(prosail_git_package_url))

# Check if the required python function is availbe: prosail.py & dataSpec_P5.csv
try:
  # Change the working directory
  os.chdir(os.path.join(original_dir, '{}'.format(folder_name)))
  import prosail
  logging.info('ProSAIL method (prosail) has successfully imported!')
except:
  logging.warning('prosail.py does not exists in {}'\
                  ' Please check from other sources!'.
                  format(os.getcwd()))
  sys.exit()

# Check if the required dataset is availbe: prosail.py & dataSpec_P5.csv
try:
  os.path.exists('dataSpec_P5.csv')
except:
  logging.warning('dataSpec_P5.csv does not exists in {}'.format(os.getcwd()))
  sys.exit()

logging.info("Initilization success. READY TO GO!")

INFO:root:work folder has been successfully created!
INFO:root:ProSAIL package has successfully cloned from https://github.com/robintw/ProSAIL.git repository
INFO:root:ProSAIL method (prosail) has successfully imported!
INFO:root:Initilization success. READY TO GO!


In [ ]:
def plotting_function(delta_LAI, N, Cab, Car, Cbrown, Cw, Cm, psoil, hspot, tts, tto, psi):
  """
  Plot the reflectance charts in different conditions, senarios.

  :param N, Cab, Car, Cbrown, Cw, Cm (LEAF CHEM & STR PROPERTIES)
  :param psoil (Soil Reflectance Properties)
  :param LAI, hspot, tts, tto, psi (4SAIL canopy structure parameters)
  :return: charts plot
  """

  fig, axs = plt.subplots(2, 2, figsize=(21,10))

  # Create a json dictionary object where store initinal forest type and LAI value
  simulation_init_info =OrderedDict({
      "Conifer_forest": {
          "TypeLidf": 2,
          "LAI": {"Spring": 3.1,
                  "Summer": 3.5,
                  "Autumn": 3.7,
                  "Winter": 3.1
              }
          },
      "Broadleaf_forest": {
          "TypeLidf": 2,
          "LAI": {"Spring": 0.5,
                  "Summer": 2.5,
                  "Autumn": 2.0,
                  "Winter": 0.0
              }
          }
      })
  
  # Some addition parameters which support the plots
  colors = ["green", "orange"]
  events = ["undisturbed", "disturbed (LAI)", "disturbed (Others)"]

  # Loop over different senarios, compute reflectance (uses PROSAIL), and plot charts
  for i, forest_type in enumerate(list(simulation_init_info.keys())):
    LIDF = simulation_init_info[forest_type]['TypeLidf']
    LAI_dict = simulation_init_info[forest_type]['LAI']
    for j, _lai in enumerate(LAI_dict.keys()):
      LAI =  LAI_dict[_lai]
      params = [N, Cab, Car, Cbrown, Cw, Cm, psoil, hspot, tts, tto, psi]
      [N_orig, Cab_orig, Car_orig, Cbrown_orig, Cw_orig, Cm_orig, psoil_orig, \
       hspot_orig, tts_orig, tto_orig, psi_orig] = params_orig

      # reflectance computation for undisturbed senarios
      results = p.run(N_orig, Cab_orig, Car_orig, Cbrown_orig, Cw_orig, Cm_orig, \
                      psoil_orig, LAI, hspot_orig, tts_orig, tto_orig, psi_orig, LIDF)
      
      if j < 2:
        j, i0 = j, 0
      else:
        j, i0 = j-2, 1

      # Start plotting for undisturbed senarios
      axs[i0, j].plot(results[0], results[1], lw=2, color=colors[i], \
                      label = '{}[{}]'.format(forest_type, events[0]))
      
      tkw = dict(size=4, width=1.5)
      axs[i0, j].set_title('{} [N_ud = {}, Cab_ud = {}, psoil_ud = {}]'.\
                           format(_lai, N_orig, Cab_orig, psoil_orig ), fontsize=16)
      axs[i0, j].set_xlabel("Wavelength [nm]", fontsize=14)
      axs[i0, 0].set_ylabel("Reflectance [-]", fontsize=14)
      axs[i0, j].tick_params(axis='y', labelsize=16, **tkw)
      axs[i0, j].tick_params(axis='x', labelsize=16, **tkw)
      # axs[i0, j].legend(fontsize=16, loc='lower center', \
      #                   fancybox=True, framealpha=0.9, ncol=2)
      axs[i0, j].legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0., fontsize=13)

      if delta_LAI !=0 and params == params_orig:
        # Reflectance computation for disturbed senarios: normal & ± uncertainty of LAI
        LAI_p = LAI + 0.01*delta_LAI*LAI
        if LAI - 0.01*delta_LAI > 0:
          LAI_n = LAI - 0.01*delta_LAI*LAI
        else:
          LAI_n = 0

        results_p = p.run(N, Cab, Car, Cbrown, Cw, Cm, \
                        psoil, LAI_p, hspot, tts, tto, psi, LIDF)
        results_n = p.run(N, Cab, Car, Cbrown, Cw, Cm, \
                        psoil, LAI_n, hspot, tts, tto, psi, LIDF)           
        # Start plotting for disturbed senarios: normal & ±10% uncertainty of LAI
        axs[i0, j].plot(results_p[0], results_p[1], '-', lw=0.75, \
                        color=colors[i], label = '{}[{}]'.format(forest_type, events[1]))
        axs[i0, j].plot(results_n[0], results_n[1], '-', lw=0.75, color=colors[i])     
        tkw = dict(size=4, width=1.5)
        axs[i0, j].set_title('{} [LAI_ud/d={}/{:.1f},{:.1f}, N_ud/d = {}/{}, Cab_ud/d = {}/{}, psoil_ud/d = {}/{}]'.
                           format(_lai, LAI, LAI_p, LAI_n, N_orig, N, Cab_orig, Car, psoil_orig, psoil), fontsize=16)
        axs[i0, j].legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0., fontsize=13)

      if params != params_orig:
        # Reflectance computation for disturbed senarios: normal & ±10% uncertainty of LAI
        results = p.run(N, Cab, Car, Cbrown, Cw, Cm, \
                        psoil, LAI, hspot, tts, tto, psi, LIDF)        
        # Start plotting for disturbed senarios: normal & ±10% uncertainty of LAI
        axs[i0, j].plot(results[0], results[1], '--', lw=1, \
                        color=colors[i], label = '{}[{}]'.format(forest_type, events[2]))    
        tkw = dict(size=4, width=1.5)
        axs[i0, j].set_title('{} [LAI_ud/d={}/{}, N_ud/d = {}/{}, Cab_ud/d = {}/{}, psoil_ud/d = {}/{}]'.
                           format(_lai, LAI, LAI, N_orig, N, Cab_orig, Car, psoil_orig, psoil), fontsize=16)
        axs[i0, j].legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0., fontsize=13)

  fig.tight_layout()
  # fig.subplots_adjust(right=0.75)
  fig.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=1.05, hspace=0.5, wspace=0.5)

In [ ]:
# Setup stype and layout for the chart
style = {'description_width': 'initial'}
layout = {'width': '600px'}

# Change/Re-assign the working directory
os.chdir(os.path.join(original_dir, '{}'.format(folder_name)))

# Call the model class
p = prosail.Prosail()

# Initinal value for input parameters undisturbed senarios.
params_orig = []

# Initinal LEAF CHEM & STR PROPERTIES parameters (values and ranges)
################################################################################
## LEAF CHEM & STR PROPERTIES	: N, Cab, Car, Cbrown, Cw, Cm                    #
################################################################################
N		= 1.5	    # structure coefficient
Cab		= 40		# chlorophyll content (µg.cm-2) 
Car		= 8		  # carotenoid content (µg.cm-2)
Cbrown	= 0.0	# brown pigment content (arbitrary units)
Cw		= 0.01	# EWT (cm)
Cm		= 0.009	# LMA (g.cm-2)
params_orig = params_orig + [N, Cab, Car, Cbrown, Cw, Cm]       
N = widgets.FloatSlider(value=N, min=1, max=3.5, step=0.1, \
                        description='N[leaf structure parameter]', \
                        style=style, layout = layout)
Cab = widgets.FloatSlider(value=Cab, min=0, max=100, step=10, \
                          description='Cab[chlorophyll a+b content (in µg/cm²)]', \
                          style=style, layout = layout)
Car = widgets.FloatSlider(value=Car, min=0, max=30, step=1, \
                          description='Car[carotenoids (carotenes + xanthophylls) content (in µg/cm²)]', \
                          style=style, layout = layout)
Cbrown = widgets.FloatSlider(value=Cbrown, min=0, max=1, step=0.1, \
                             description='Cbrown[brown pigments content (in arbitrary units)]', \
                             style=style, layout = layout)
Cw = widgets.FloatSlider(value=Cw, min=0.00005, max=0.05, step=0.001, \
                         description='Cw[equivalent water thickness (in g/cm² or cm)]', \
                         style=style, layout = layout)
Cm = widgets.FloatSlider(value=Cm, min=0.002, max=0.020, step=0.001, \
                         description='Cm[dry matter content (in g/cm²)]', \
                         style=style, layout = layout)

# Initinal Soil Reflectance Properties parameter (value and range)
################################################################################
##	Soil Reflectance Properties: psoil	                                       #
################################################################################
psoil	= 0.5	  	# soil factor (psoil=0: wet soil / psoil=1: dry soil)
params_orig = params_orig + [psoil]
psoil = widgets.FloatSlider(value=psoil, min=0, max=1, step=0.1, \
                            description='psoil[soil factor]', \
                            style=style, layout = layout)

# Initinal 4SAIL canopy structure parameters (values and ranges)
################################################################################
## 4SAIL canopy structure parameters: LAI, hspot, tts, tto, psi	               #
################################################################################
# LAI		    =	5.   	# leaf area index (m^2/m^2)
delta_LAI		=	10   	# Uncertanty level (in %) of leaf area index (m^2/m^2)
hspot	    =	0.01  # hot spot
tts		    =	35.	  # solar zenith angle (°)
tto		    =	0.	  # observer zenith angle (°)
psi		    =	90.   # azimuth (°)
params_orig = params_orig  + [hspot, tts, tto, psi]
delta_LAI = widgets.FloatSlider(value=delta_LAI, min=0, max=100, step=10, \
                        description='delta_LAI[uncertanty level of leaf area index (%)]', \
                        style=style, layout = layout)
hspot = widgets.FloatSlider(value=hspot, min=0.005, max=0.2, step=0.01, \
                            description='hspot[hot spot]', \
                            style=style, layout = layout)
tts = widgets.FloatSlider(value=tts, min=0, max=90, step=10, \
                          description='tts[solar zenith angle (°)]', \
                          style=style, layout = layout)
tto = widgets.FloatSlider(value=tto, min=0, max=90, step=10, \
                          description='tto[observer zenith angle (°)]', \
                          style=style, layout = layout)
psi = widgets.FloatSlider(value=psi, min=0, max=180, step=10, \
                          description='psi[azimuth (°)]', \
                          style=style, layout = layout)

# Call plotting_function function and present the result
ouput = widgets.interactive(plotting_function, \
                            delta_LAI=delta_LAI, N=N, Cab=Cab, Car=Car, Cbrown=Cbrown, Cw=Cw, Cm=Cm, \
                            psoil=psoil, hspot=hspot, tts=tts, tto=tto, psi=psi)
display(ouput)

interactive(children=(FloatSlider(value=10.0, description='delta_LAI[uncertanty level of leaf area index (%)]'…